## 필요한 모듈 불러오기

In [2]:
import numpy as np
import pandas as pd
import json
from gensim.models import Word2Vec
from scipy.sparse import csr_matrix
import os; os.environ['KMP_DUPLICATE_LIB_OK']='True'
from collections import Counter
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
mpl.pyplot.rc('font', family='Malgun Gothic')
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('ggplot')

## 태그 정보가 담긴 데이터셋 불러오기

In [3]:
playlists_df = pd.read_json('train.json', typ = 'frame')
# playlists_df.head()

In [4]:
playlist_id_tags = playlists_df[['id', 'tags']]
playlist_id_tags
tag_copy = playlist_id_tags.copy()
# 각 플레이리스트에는 1개부터 n개 까지 여러개의 태그가 붙어져 있다.

In [5]:
# 각 id 별로 태그 개수만큼 조인
playlist_id_tags_unnest = np.dstack(
    (
        np.repeat(playlist_id_tags.id.values, list(map(len, playlist_id_tags.tags))), 
        np.concatenate(playlist_id_tags.tags.values)
    ))

In [6]:
playlist_id_tags_unnest[0]

array([['61281', '락'],
       ['10532', '추억'],
       ['10532', '회상'],
       ...,
       ['100389', '팝송추천'],
       ['100389', '팝송'],
       ['100389', '팝송모음']], dtype='<U21')

In [7]:
playlist_id_tags = pd.DataFrame(data=playlist_id_tags_unnest[0], columns=playlist_id_tags.columns)
playlist_id_tags['id'] = playlist_id_tags['id'].astype(str)
playlist_id_tags.head()

,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한


## 각 플레이리스트의 태그 정보에 word2vec 적용

In [8]:
# 1. 활용할 태그 데이터 준비
df = playlist_id_tags['tags'].tolist()
tags_dataset = tag_copy['tags'].tolist()

In [10]:
# 2. 태그 -> 벡터화 모델 적용
model = Word2Vec(tags_dataset, sg=1, window=3, min_count=1)
model

In [11]:
# 3. init_sims 명령으로 필요없는 메모리를 unload
model.init_sims(replace=True)

In [15]:
# # 최종 모델 저장
# model.save('tag2vec.model')
# # 저장한 모델 불러오기
# model = Word2Vec.load('tag2vec.model')

In [17]:
# '여름' 이라는 태그 예시 적용 
model.wv.similar_by_word(word='여름', topn=10)

[('휴양지', 0.891598105430603),
 ('시원', 0.8818683624267578),
 ('청량', 0.8792890310287476),
 ('시원한', 0.874660074710846),
 ('트로피컬', 0.8719056844711304),
 ('바닷가', 0.8544214963912964),
 ('Summer', 0.8534398078918457),
 ('청량함', 0.8504664897918701),
 ('바다', 0.848107099533081),
 ('더위', 0.8472164869308472)]

## 벡터화 된 태그들의 상관관계를 보기위해 T-SNE 시각화 적용 

In [ ]:
from sklearn.manifold import TSNE

# 벡터화 된 태그들을 TSNE로 시각화 적용
sim_tags = list(model.wv.vocab)

X = model[sim_tags]
tsne = TSNE(n_components=2, verbose=1)
X_tsne = tsne.fit_transform(X)
X_tsne.shape

In [ ]:
# T-SNE를 시각화
df_subset = pd.DataFrame()
df_subset['tsne-2d-x'] = X_tsne[:,0]
df_subset['tsne-2d-y'] = X_tsne[:,1]

plt.figure(figsize=(40,20))

sns.scatterplot(
    x="tsne-2d-x", y="tsne-2d-y",
    hue="tsne-2d-y",
    #palette=sns.color_palette("muted"),
    data=df_subset,
    legend="full",
    alpha=0.3
)
# for word, pos in df.iterrows():
#     ax.annotate(word, pos, fontsize=30)
plt.show()

# # 시간 절약을 위한 새로운 파일 생성
# df_subset.to_csv('df_subset.csv', index=True)